# Capstone 1: Data Wrangling
## By: Sinead O'Connor

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from bs4 import BeautifulSoup
import requests
from datetime import datetime
from datetime import timedelta
from fractions import Fraction
from urllib.request import urlretrieve
import pickle

First, I will create a function to import game logs from Retrosheet.org and create a dataframe.  You can specify the seasons you want to import. Information about each feature of the gamelogs can be found at https://www.retrosheet.org/gamelogs/glfields.txt.
 
 The information used here was obtained free of charge from and is copyrighted by Retrosheet.  Interested parties may  contact Retrosheet at "www.retrosheet.org".

I will then create a function to import starting pitching logs from www.baseballmusings.com.

In [2]:
def import_RSgamelogs(firstseason, lastseason):
    """Import game logs for specified seasons from Retrosheet.org and save them in a DataFrame"""
    #initialize 
    seasonfiles = []
    
    for season in range(firstseason, lastseason+1):
        season_url = 'https://www.retrosheet.org/gamelogs/gl' + str(season) + '.zip'
        seasonfile = 'gamelogs' + str(season) + '.txt'
        urlretrieve(season_url, seasonfile)
        seasonfiles.append(seasonfile)
        
    seasonlogs = [pd.read_csv(file, header=None, encoding="utf-8", compression = 'zip') for file in seasonfiles]
    gamelogs = pd.concat(seasonlogs, ignore_index=True, axis=0)
    
    #add column names
    headers_url = 'https://raw.githubusercontent.com/maxtoki/baseball_R/master/data/game_log_header.csv'
    urlretrieve(headers_url, 'headers.csv')
    headers = pd.read_csv('headers.csv')
    gamelogs.columns = headers.columns
    gamelogs.rename(columns = {'Attendence': 'Attendance'}, inplace=True)
    
    return gamelogs



In [3]:
def import_BMpitchlogs(firstseason, lastseason):
    """Import starting pitching logs from www.baseballmusings.com"""
    teampitchlogs = []
    start_urls = []
    
    #get urls for each MLB team starting pitcher logs for all seasons
    # from firstseason to lastseson
    for team in range(1,31):
        start_urls.append('https://www.baseballmusings.com/cgi-bin/StartingInfo.py?StartDate=1%2F1%2F'+str(firstseason) + '&EndDate=12%2F31%2F' + str(lastseason) + '&GameType=all&PlayedVs=0&Park=0&PlayerID='+str(team))
    
    #for each url (ie each team) get the table with the pitching info for each game
    for team in start_urls:
        r = requests.get(team)
        html_doc = r.text
        soup = BeautifulSoup(html_doc, 'html.parser')
        table = soup.find('table', attrs = {'class': 'dbd'})
        header = table.find('tr')
        
        #get the column names
        columns = [col.get_text() for col in header.find_all('th')]
        
        #get the stats for the starting pitcher of each game
        games = []
        rows = table.find_all('tr', attrs = {'class': not 'toprow'})
        for game in rows:  #since the last row of each table just has aggregate data
            td = game.find_all('td')
            pitchstats = [stat.get_text() for stat in td if td[0] != 'Total']
            games.append(pitchstats)
        
        teampitchlog = pd.DataFrame(games)
        teampitchlogs.append(teampitchlog)

    
    #concatenate the starting pitcher logs for all the teams
    pitchlogs = pd.concat(teampitchlogs, ignore_index=True)
    pitchlogs.columns = columns
    return pitchlogs

In [4]:
firstseason = 2009
lastseason = 2018

In [5]:
RSgamelogs = import_RSgamelogs(firstseason, lastseason)

In [6]:

RSgamelogs.head()

,Date,DoubleHeader,DayOfWeek,VisitingTeam,VisitingTeamLeague,VisitingTeamGameNumber,HomeTeam,HomeTeamLeague,HomeTeamGameNumber,VisitorRunsScored,...,HomeBatting7Name,HomeBatting7Position,HomeBatting8PlayerID,HomeBatting8Name,HomeBatting8Position,HomeBatting9PlayerID,HomeBatting9Name,HomeBatting9Position,AdditionalInfo,AcquisitionInfo
0,20090405,0,Sun,ATL,NL,1,PHI,NL,1,4,...,Pedro Feliz,5,ruizc001,Carlos Ruiz,2,myerb001,Brett Myers,1,NaN,Y
1,20090406,0,Mon,COL,NL,1,ARI,NL,1,8,...,Conor Jackson,7,snydc002,Chris Snyder,2,webbb001,Brandon Webb,1,NaN,Y
2,20090406,0,Mon,NYN,NL,1,CIN,NL,1,2,...,Ramon Hernandez,2,gonza002,Alex Gonzalez,6,haraa001,Aaron Harang,1,NaN,Y
3,20090406,0,Mon,WAS,NL,1,FLO,NL,1,6,...,Cody Ross,9,maybc001,Cameron Maybin,8,nolar001,Ricky Nolasco,1,NaN,Y
4,20090406,0,Mon,CHN,NL,1,HOU,NL,1,4,...,Hunter Pence,9,bourm001,Michael Bourn,8,oswar001,Roy Oswalt,1,NaN,Y


In [7]:
pitchlogs = import_BMpitchlogs(firstseason, lastseason)

In [8]:
pitchlogs.head()

,Game Date,Pitcher,Opponent,GS,CG,GF,W,L,Sv,Sho,IP,H,R,ER,HR,BB,K,HBP,BK,WP
0,04/06/2009\n,Joe Saunders,Vs. OAK,1,0,0,1,0,0,0,6 2/3,3,0,0,0,2\n,2,1,0,0
1,04/07/2009\n,Dustin Moseley,Vs. OAK,1,0,0,0,0,0,0,6,9,3,3,0,0\n,4,0,0,0
2,04/08/2009\n,Nick Adenhart,Vs. OAK,1,0,0,0,0,0,0,6,7,0,0,0,3\n,5,0,0,1
3,04/10/2009\n,Jered Weaver,Vs. BOS,1,0,0,1,0,0,0,6 2/3,4,1,0,0,2\n,8,0,0,0
4,04/11/2009\n,Joe Saunders,Vs. BOS,1,0,0,0,1,0,0,7,7,4,4,2,3\n,2,0,0,0


I will now clean each data set.  First I will clean each dataframe and then join them together and clean the joined set. 

## Identifying Missing Values & Cleaning- GameLogs DataFrame

In [9]:
RSgamelogs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24298 entries, 0 to 24297
Columns: 161 entries, Date to AcquisitionInfo
dtypes: float64(2), int64(82), object(77)
memory usage: 29.8+ MB


I will first convert the date to a datetime object.

In [10]:
RSgamelogs['Date'] = pd.to_datetime(RSgamelogs['Date'].astype(str), infer_datetime_format=True)

#check that it was successfully converted to datetime
RSgamelogs['Date'].head()

0   2009-04-05
1   2009-04-06
2   2009-04-06
3   2009-04-06
4   2009-04-06
Name: Date, dtype: datetime64[ns]

In [11]:
RSgamelogs['AcquisitionInfo'].unique()

array(['Y'], dtype=object)

AcquisitionInfo is 'Y' for all each game, meaning that Retrosheet had information for the complete game.  So missing values should just mean that it did not apply to that particular game.  I will explore  missing values further.  As noted on Retrosheet.org at https://www.retrosheet.org/gamelogs/glfields.txt, in some fields such as Umpire IDs and Umpire Names, and 

In [12]:
nullcounts = RSgamelogs.isnull().sum().sort_values(ascending=False)
nullcounts[nullcounts>0]

ForfeitInfo           24298
UmpireLFID            24294
UmpireRFID            24294
ProtestInfo           24285
CompletionInfo        24280
AdditionalInfo        24165
SavingPitcherID       11867
GameWinningRBIID       1162
Umpire2BID               24
Attendance                2
LosingPitcherNAme         1
LosingPitcherID           1
WinningPitcherName        1
WinningPitcherID          1
dtype: int64

In [13]:
protests = RSgamelogs[RSgamelogs['ProtestInfo'].notnull()]
protests[['Date', 'HomeTeam', 'VisitingTeam', 'HomeRunsScore', 'VisitorRunsScored', 'ProtestInfo']]
#             "P" -- the game was protested by an unidentified team
#             "V" -- a disallowed protest was made by the visiting team
#             "H" -- a disallowed protest was made by the home team
#             "X" -- an upheld protest was made by the visiting team
#             "Y" -- an upheld protest was made by the home team

,Date,HomeTeam,VisitingTeam,HomeRunsScore,VisitorRunsScored,ProtestInfo
1020,2009-06-21,FLO,NYA,6,5,V
3007,2010-05-18,NYA,BOS,6,7,H
3154,2010-05-28,SDN,WAS,3,5,V
4159,2010-08-13,SFN,SDN,2,3,V
6936,2011-09-04,FLO,PHI,5,4,V
8864,2012-08-03,CHA,ANA,8,6,V
14018,2014-08-19,CHN,SFN,2,1,X
14072,2014-08-23,TOR,TBA,5,4,V
14136,2014-08-28,ANA,OAK,4,3,V
15325,2015-05-31,SEA,CLE,3,6,V


There has only been one upheld protest in the 2009-2018 season of MLB.

For some of these variables, you would expect them to have corresponding variables, with the same number of null values, for example, if SavingPitcherID has 11,867 null values so should SavingPitcherName, so it is odd that it has 0 null values.  Reading the information about each variable on https://www.retrosheet.org/gamelogs/glfields.txt, reveals that certain fields will be filled with an empty string or '(none)', if it did not apply to that particular game. For example it says, "If any umpire positions were not filled for a particular game the fields will be "","(none)"." So I will replace empty strings and "(none)" with NaN.

In [14]:
RSgamelogs.replace('', np.nan, inplace=True)
RSgamelogs.replace('(none)', np.nan, inplace=True)
new_nullcounts = RSgamelogs.isnull().sum().sort_values(ascending=False)
new_nullcounts[new_nullcounts>0]

ForfeitInfo           24298
UmpireLFID            24294
UmpireRFName          24294
UmpireRFID            24294
UmpireLFName          24294
ProtestInfo           24285
CompletionInfo        24280
AdditionalInfo        24165
SavingPitcherName     11867
SavingPitcherID       11867
GameWinningRBIName     1162
GameWinningRBIID       1162
Umpire2BName             24
Umpire2BID               24
Attendance                2
LosingPitcherNAme         1
LosingPitcherID           1
WinningPitcherName        1
WinningPitcherID          1
dtype: int64

Now the null value counts make more sense.

Missing Value Analysis:

We see that no games were Forfeited in the 2009-2018 regular season.  


In [15]:
missingPitch = RSgamelogs[RSgamelogs['WinningPitcherID'].isnull()]
missingPitch[['Date','LosingPitcherNAme', 'LosingPitcherID', 'WinningPitcherID',  'WinningPitcherID']]

,Date,LosingPitcherNAme,LosingPitcherID,WinningPitcherID,WinningPitcherID
19387,2016-09-29,NaN,NaN,NaN,NaN


We see that the null values for LosingPitcherName, LosingPitcherID, WinningPitcherID, and WinningPitcherID all occur in the same game.

In [16]:
missingPitch[['Date','HomeTeam','VisitingTeam','HomeRunsScore','VisitorRunsScored']]

,Date,HomeTeam,VisitingTeam,HomeRunsScore,VisitorRunsScored
19387,2016-09-29,PIT,CHN,1,1


Looking at the score for the game, we see that the game resulted in a tie, so it is logical that there is no winning or losing pitcher.  In fact, a Google search reveals that the September 29, 2016 game between the Pittsburg Pirates and the Chicago Cubs ended in a tie due to the rain.  https://www.youtube.com/watch?v=zr4zvsVQvk4

In [17]:
missingAtt = RSgamelogs[RSgamelogs['Attendance'].isnull()]
missingAtt.iloc[:, :20]

,Date,DoubleHeader,DayOfWeek,VisitingTeam,VisitingTeamLeague,VisitingTeamGameNumber,HomeTeam,HomeTeamLeague,HomeTeamGameNumber,VisitorRunsScored,HomeRunsScore,LengthInOuts,DayNight,CompletionInfo,ForfeitInfo,ProtestInfo,ParkID,Attendance,Duration,VisitorLineScore
14884,2015-04-29,0,Wed,CHA,AL,18,BAL,AL,20,2,8,51,D,NaN,NaN,NaN,BAL12,NaN,123,000020000
20508,2017-06-21,0,Wed,ANA,AL,75,NYA,AL,69,4,8,51,N,NaN,NaN,NaN,NYC21,NaN,194,000200002


A Google search reveals that the April 29, 2015 game between the Baltimore Orioles and Chicago White Sox was the first ever crowdless MLB baseball game.  At the time there was civil unrest in Baltimore as a result of police brutality which led to the death of Freddie Gray, a 25-year-old African American.  https://www.youtube.com/watch?v=L9iXFsCFVBs&t=37s.  I will use an average instead.

However, the June 21, 2017 game between the New York Yankees and the Los Angeles Angels did have a crowd.  According to espn.com, the attendance was 39,911. 

In [18]:
zeroAtt = RSgamelogs[RSgamelogs['Attendance'] == 0]
print(zeroAtt.shape)
zeroAtt.head()

(88, 161)


,Date,DoubleHeader,DayOfWeek,VisitingTeam,VisitingTeamLeague,VisitingTeamGameNumber,HomeTeam,HomeTeamLeague,HomeTeamGameNumber,VisitorRunsScored,...,HomeBatting7Name,HomeBatting7Position,HomeBatting8PlayerID,HomeBatting8Name,HomeBatting8Position,HomeBatting9PlayerID,HomeBatting9Name,HomeBatting9Position,AdditionalInfo,AcquisitionInfo
294,2009-04-28,1,Tue,SEA,AL,20,CHA,AL,19,1,...,Alexei Ramirez,6,millc001,Corky Miller,2,lillb001,Brent Lillibridge,4,NaN,Y
583,2009-05-20,1,Wed,ARI,NL,39,FLO,NL,40,6,...,Ronny Paulino,2,coghc001,Chris Coghlan,7,volsc001,Chris Volstad,1,NaN,Y
719,2009-05-29,1,Fri,OAK,AL,45,TEX,AL,47,3,...,Chris Davis,3,saltj001,Jarrod Saltalamacchia,2,andre001,Elvis Andrus,6,NaN,Y
794,2009-06-04,1,Thu,SFN,NL,51,WAS,NL,52,5,...,Ronnie Belliard,4,nievw001,Wil Nieves,2,zimmj003,Jordan Zimmermann,1,NaN,Y
1393,2009-07-21,1,Tue,ANA,AL,91,KCA,AL,92,8,...,Alberto Callaspo,4,olivm001,Miguel Olivo,2,betay001,Yuniesky Betancourt,6,NaN,Y


We also see that there are 88 games where attendance is 0, so I will replace these with NaNs.

In [19]:
RSgamelogs['Attendance'].replace(0, np.nan, inplace=True)

Since we know the attendance at Yankee Stadium on June 21, 2017 to have been 39,911, I will use that as the new value.

For the other 89 games that are missing attendance information, I will use the average for that season at the home field.

In [20]:
RSgamelogs.loc[((RSgamelogs['Date'] == '2017-06-21') & (RSgamelogs['HomeTeam'] == 'NYA')),'Attendance'] = 39911

In [21]:
RSgamelogs.loc[RSgamelogs['Attendance'].isnull(),'Attendance'] = RSgamelogs.groupby(['HomeTeam', RSgamelogs['Date'].dt.year])['Attendance'].transform(lambda x: x.fillna(x.mean()))

In [22]:
RSgamelogs[RSgamelogs['Attendance'].isnull()]

,Date,DoubleHeader,DayOfWeek,VisitingTeam,VisitingTeamLeague,VisitingTeamGameNumber,HomeTeam,HomeTeamLeague,HomeTeamGameNumber,VisitorRunsScored,...,HomeBatting7Name,HomeBatting7Position,HomeBatting8PlayerID,HomeBatting8Name,HomeBatting8Position,HomeBatting9PlayerID,HomeBatting9Name,HomeBatting9Position,AdditionalInfo,AcquisitionInfo


Now there is no null attendance.

I will also convert the date to a datetime object.

## Identify Categorical Variables - RetroSheet GameLogs DataFrame

In [23]:
#identify categorical variables
unique_vals = [[col, RSgamelogs[col].nunique()] for col in RSgamelogs.columns if RSgamelogs[col].nunique() <= 35]
unique_vals.sort(key = lambda ls: ls[1])
print(unique_vals)

[['ForfeitInfo', 0], ['AcquisitionInfo', 1], ['VisitingTeamLeague', 2], ['HomeTeamLeague', 2], ['DayNight', 2], ['VisitorTP', 2], ['HomeTP', 2], ['DoubleHeader', 3], ['ProtestInfo', 3], ['VisitorCI', 3], ['HomeCI', 3], ['HomeBalks', 3], ['VisitorBalks', 4], ['UmpireLFID', 4], ['UmpireLFName', 4], ['UmpireRFID', 4], ['UmpireRFName', 4], ['VisitorSF', 5], ['VisitorIBB', 5], ['VisitorCS', 5], ['VisitorPassed', 5], ['HomeSH', 5], ['HomeSF', 5], ['HomeCS', 5], ['HomePassed', 5], ['VisitorT', 6], ['VisitorSH', 6], ['VisitorHBP', 6], ['HomeT', 6], ['HomeIBB', 6], ['DayOfWeek', 7], ['VisitorGDP', 7], ['VisitorWP', 7], ['VisitorE', 7], ['HomeHBP', 7], ['HomeGDP', 7], ['HomeWP', 7], ['HomeDB', 7], ['VisitorDB', 8], ['HomeE', 8], ['VisitorHR', 9], ['VisitorSB', 9], ['HomeHR', 9], ['HomeSB', 9], ['VisitorBatting1Position', 9], ['VisitorBatting2Position', 9], ['VisitorBatting3Position', 9], ['VisitorBatting4Position', 9], ['VisitorBatting5Position', 9], ['VisitorBatting6Position', 9], ['VisitorBatt

Based on the above data, the columns that contain categorical variables are VisitorTeamLeague (AL/NL), HomeTeamLeague, DayOfWeek, DayNight, ProtestInfo, and the Visitor and Home Positions for each player.  Positions are 1-10 (1: pitcher, 2: catcher, 3: first base, ..., 10: designated hitter).

It is odd that there are 31 teams since there are only 30 MLB teams.

In [24]:
RSgamelogs['HomeTeam'].unique()

array(['PHI', 'ARI', 'CIN', 'FLO', 'HOU', 'SDN', 'SLN', 'ANA', 'BAL',
       'MIN', 'TEX', 'TOR', 'SFN', 'BOS', 'CHA', 'ATL', 'COL', 'MIL',
       'CLE', 'DET', 'KCA', 'OAK', 'CHN', 'LAN', 'NYN', 'PIT', 'WAS',
       'TBA', 'SEA', 'NYA', 'MIA'], dtype=object)

This is because the Florida Marlins (FLO) became the Miami Marlins (MIA) in 2012. I will change the MIA to FLO.

In [25]:
RSgamelogs.replace('MIA','FLO', inplace=True)

In [26]:
print(RSgamelogs['HomeTeam'].nunique())
print(RSgamelogs['VisitingTeam'].unique())

30
['ATL' 'COL' 'NYN' 'WAS' 'CHN' 'LAN' 'PIT' 'OAK' 'NYA' 'SEA' 'CLE' 'DET'
 'MIL' 'TBA' 'KCA' 'PHI' 'SFN' 'HOU' 'BOS' 'MIN' 'TOR' 'TEX' 'SLN' 'CIN'
 'SDN' 'CHA' 'BAL' 'FLO' 'ANA' 'ARI']


## Identifying Missing Values & Cleaning- Starting Pitching DataFrame

In [27]:
pitchlogs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48596 entries, 0 to 48595
Data columns (total 20 columns):
Game Date    48596 non-null object
Pitcher      48596 non-null object
Opponent     48596 non-null object
GS           48596 non-null object
CG           48596 non-null object
GF           48596 non-null object
W            48596 non-null object
L            48596 non-null object
Sv           48596 non-null object
Sho          48596 non-null object
IP           48596 non-null object
H            48596 non-null object
R            48596 non-null object
ER           48596 non-null object
HR           48596 non-null object
BB           48596 non-null object
K            48596 non-null object
HBP          48596 non-null object
BK           48596 non-null object
WP           48596 non-null object
dtypes: object(20)
memory usage: 7.4+ MB


There are no null values for in the starting pitching data.  It also makes sence that there are 48,596 rows (2 * 24298 - the number of rows in the game logs) since there are two starting pitchers for each game (the home team starter and the visiting team starter).

The data set is relatively clean but we need to fix some things.


## Cleaning the Starting Pitcher Logs

In [28]:
pitchlogs['BB'].unique()

array(['2\n', '0\n', '3\n', '1\n', '4\n', '5\n', '6\n', '7\n', '8\n',
       '9\n'], dtype=object)

For some reason, all entries in the 'BB' column have a \n at the end, so we will need to strip that.

All of the game dates also end in '\n' or in '\n(1)' or '\n(2)' if more than one game was played by a team on the same day, so I will extract just the first 10 characters of the string which contains the date in m/d/Y fomat.

In [29]:
#clean 'Game Date'
pitchlogs['Game Date'] = pitchlogs['Game Date'].str[:10]

#convert 'Game Date' to date time object
pitchlogs['Game Date'] = pd.to_datetime(pitchlogs['Game Date'], infer_datetime_format=True)

#remove '\n' from 'BB' column
pitchlogs['BB'] = pitchlogs['BB'].str.rstrip('\n')
pitchlogs.head()


,Game Date,Pitcher,Opponent,GS,CG,GF,W,L,Sv,Sho,IP,H,R,ER,HR,BB,K,HBP,BK,WP
0,2009-04-06,Joe Saunders,Vs. OAK,1,0,0,1,0,0,0,6 2/3,3,0,0,0,2,2,1,0,0
1,2009-04-07,Dustin Moseley,Vs. OAK,1,0,0,0,0,0,0,6,9,3,3,0,0,4,0,0,0
2,2009-04-08,Nick Adenhart,Vs. OAK,1,0,0,0,0,0,0,6,7,0,0,0,3,5,0,0,1
3,2009-04-10,Jered Weaver,Vs. BOS,1,0,0,1,0,0,0,6 2/3,4,1,0,0,2,8,0,0,0
4,2009-04-11,Joe Saunders,Vs. BOS,1,0,0,0,1,0,0,7,7,4,4,2,3,2,0,0,0


In [30]:
pitchlogs = pitchlogs.sort_values('Game Date')
pitchlogs.head()

,Game Date,Pitcher,Opponent,GS,CG,GF,W,L,Sv,Sho,IP,H,R,ER,HR,BB,K,HBP,BK,WP
40498,2009-04-05,Brett Myers,Vs. ATL,1,0,0,0,1,0,0,6,8,4,4,3,1,6,0,0,0
24300,2009-04-05,Derek Lowe,At PHI,1,0,0,1,0,0,0,8,2,0,0,0,0,4,0,0,0
0,2009-04-06,Joe Saunders,Vs. OAK,1,0,0,1,0,0,0,6 2/3,3,0,0,0,2,2,1,0,0
42118,2009-04-06,Paul Maholm,At SLN,1,0,0,0,0,0,0,6 2/3,7,2,1,0,1,1,1,0,1
38878,2009-04-06,Johan Santana,At CIN,1,0,0,1,0,0,0,5 2/3,3,1,1,0,4,7,0,0,0


## Indentifying Categorical Variables - Starting Pitcher Logs

The categorical variables will be Game Started (GS), Complete Game (CG), Game Finished (GF), Win (W), Loss (L), Save (SV), and Shutout (Sho).  These all have 1- if yes and 0- if no.  

Innings Pitched (IP) will need to be converted from a string of mixed numbers to floats and the remaining columns of stats will need to be converted to integers.

In [31]:
#convert Innings Pitched to float
def mixed_to_float(mixed):
    parts = [Fraction(part) for part in mixed.split()]
    float_num = float(sum(parts))
    return float_num

pitchlogs['IP'] = pitchlogs['IP'].apply(mixed_to_float)
pitchlogs['IP'].head()

40498    6.000000
24300    8.000000
0        6.666667
42118    6.666667
38878    5.666667
Name: IP, dtype: float64

In [32]:
numeric_cols = ['IP','H','R','ER','HR','BB','K','HBP','BK','WP']
pitchlogs[numeric_cols] = pitchlogs[numeric_cols].apply(pd.to_numeric)

In [33]:
pitchlogs.head()

,Game Date,Pitcher,Opponent,GS,CG,GF,W,L,Sv,Sho,IP,H,R,ER,HR,BB,K,HBP,BK,WP
40498,2009-04-05,Brett Myers,Vs. ATL,1,0,0,0,1,0,0,6.000000,8,4,4,3,1,6,0,0,0
24300,2009-04-05,Derek Lowe,At PHI,1,0,0,1,0,0,0,8.000000,2,0,0,0,0,4,0,0,0
0,2009-04-06,Joe Saunders,Vs. OAK,1,0,0,1,0,0,0,6.666667,3,0,0,0,2,2,1,0,0
42118,2009-04-06,Paul Maholm,At SLN,1,0,0,0,0,0,0,6.666667,7,2,1,0,1,1,1,0,1
38878,2009-04-06,Johan Santana,At CIN,1,0,0,1,0,0,0,5.666667,3,1,1,0,4,7,0,0,0


## Merge the Game Logs with the Pitch Logs

I will use a left join since I want to match the pitchers with the home pitchers and away pitchers of each team.

In [34]:
#RetroSheet gamelogs merged with detailed info about home starting pitcher
gamelogs_hsp = pd.merge(RSgamelogs, pitchlogs, how='left', left_on = ['Date','HomeStartingPitcherName'], right_on = ['Game Date','Pitcher'], suffixes= ['_g', '_pH'])
nullvals = gamelogs_hsp.isnull().sum()
nullvals[nullvals>0]


CompletionInfo        24280
ForfeitInfo           24298
ProtestInfo           24285
Umpire2BID               24
Umpire2BName             24
UmpireLFID            24294
UmpireLFName          24294
UmpireRFID            24294
UmpireRFName          24294
WinningPitcherID          1
WinningPitcherName        1
LosingPitcherID           1
LosingPitcherNAme         1
SavingPitcherID       11867
SavingPitcherName     11867
GameWinningRBIID       1162
GameWinningRBIName     1162
AdditionalInfo        24165
Game Date              1225
Pitcher                1225
Opponent               1225
GS                     1225
CG                     1225
GF                     1225
W                      1225
L                      1225
Sv                     1225
Sho                    1225
IP                     1225
H                      1225
R                      1225
ER                     1225
HR                     1225
BB                     1225
K                      1225
HBP                 

In [35]:
gamelogs_hsp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24298 entries, 0 to 24297
Columns: 181 entries, Date to WP
dtypes: datetime64[ns](2), float64(12), int64(81), object(86)
memory usage: 33.7+ MB


Let's investigate why this results in so 1223 null values for all of the columns from the pitchlog dataframe.

In [36]:
nullrows = gamelogs_hsp[gamelogs_hsp['K'].isnull()]
nullrows[['Date','HomeStartingPitcherName','Pitcher','HomeTeam','VisitingTeam']].head()

,Date,HomeStartingPitcherName,Pitcher,HomeTeam,VisitingTeam
29,2009-04-08,Walter Silva,NaN,SDN,LAN
68,2009-04-11,David Bush,NaN,MIL,CHN
289,2009-04-28,David Bush,NaN,MIL,PIT
356,2009-05-03,David Bush,NaN,MIL,ARI
429,2009-05-08,David Bush,NaN,MIL,CHN


In [37]:
print(nullrows['HomeStartingPitcherName'].nunique())
nullrows['HomeStartingPitcherName'].unique()

67


array(['Walter Silva', 'David Bush', 'Jon Niese', 'Lucas French',
       'Rick van den Hurk', 'Rob Tejeda', 'Clayton Mortensen',
       'Andy Oliver', 'Yuniesky Maya', 'Zack Britton', 'Danny Duffy',
       'Josh Collmenter', 'Zach Stewart', 'Rubby de la Rosa',
       'Zach McAllister', 'Chris Schwinden', 'Joe Wieland',
       'Cole De Vries', 'Will Smith', 'Joe Kelly', 'Miguel Gonzalez',
       'Samuel Deduno', 'Dan Straily', 'Steve Johnson', 'Chris Rusin',
       'Tom Koehler', 'Josh Stinson', 'Brad Peacock', 'Nick Tepesch',
       'Jose Fernandez', 'Phil Irwin', 'Tony Cingrani', 'Matt Magill',
       'Josh Lindblom', 'Nathan Karns', 'Robbie Erlin', 'Alex Colome',
       'Johnny Hellweg', 'Charlie Leesman', 'Kris Johnson',
       'Matt Shoemaker', 'Trevor Cahill', 'Robbie Ross', 'Roenis Elias',
       'Mike Bolsinger', 'TJ House', 'Yohan Pino', 'Wade Matthew LeBlanc',
       'Lisalverto Bonilla', 'Nick Tropeano', 'Mike Montgomery',
       'Manny Banuelos', 'Zack Godley', 'Jon Gray', '

Looking back at the BaseballMusings tables we see that David Bush is listed as Dave Bush, and most of these seem like they could be problems with nicknames, so let's instead try to match on just the first initial and last name and see if this makes a difference. 


def abbrev_name(fullname): <br>
    beg_firstname = beg_firstname[:3] <br>
    lastname = fullname.split()[-1] <br>
    abbrev_name = first_initial + ' ' + lastname <br>
    return abbrev_name <br>
    
Using this function results in 19 unique pitchers whose abbreviated names result in null values when merged.
The pitchers are : ['Walter Silva' 'Rick van den Hurk' 'Cole De Vries' 'Tom Koehler' 'Tony Cingrani'
 'Trevor Cahill' 'Roenis Elias' 'TJ House' 'Yohan Pino' 'Jason Vargas' 'Lucas French' 'Joe Wieland' 'Joe Kelly' 'Mike Bolsinger' 'Mike Montgomery' 'Mike Clevinger' 'Joe Biagini' 'Lucas Sims' 'Mike Soroka' 'Daniel Ponce de Leon']
 
 Investingating further: <br>
 Retrosheet Game Logs--BaseballMusings Equivalent <br>
 Walter Silva--Walter Walter <br>
 Rick van den Hurk--Rick VandenHurk <br>
 Cole De Vries--Cole DeVries <br>
 Daniel Ponce de Leon - Daniel Poncedeleon <br>
 Tom Koehler--Thomas Koehler <br>
 Tony Cingrani--Anthony Cingrani <br>
 Trevor Cahill--Trevor Cahill - but the game is listed as occuring on 3/22/2014 instead of 3/23/2014 since it was played in Australia so the 23rd was the local time <br>
 Roenis Elias--Elias Roenis (wrong name) <br>
 TJ House-- T.J. House <br>
 Yohan Pino--Johan Pino (wrong spelling) <br>
 Jason Vargas--Jason Vargas - but the game was listed as 9/12/2018 on instead of 9/13/2018.  The game was                           postponed to 9/13/2018. <br>
 Mike(s)--Michael(s)
 Joe(s)--Joseph(s)
 Lucas--Luke
 I will now correct these issues and rerun the function.

In [38]:
#change the names in the pitch logs so they match the names in the RetroSheet game logs
correctnames = {'Walter Walter': 'Walter Silva', 'Rick VandenHurk': 'Rick van den Hurk', 'Cole DeVries': 'Cole De Vries', 'Daniel Poncedeleon':'Daniel Ponce de Leon','Thomas Koehler': 'Tom Koehler','Anthony Cingrani': 'Tony Cingrani', 'Elias Roenis': 'Roenis Elias', 'T.J. House':'TJ House', 'Johan Pino': 'Yohan Pino', 'Luke French':'Lucas French', 'Joseph Wieland':'Joe Wieland', 'Joseph Kelly':'Joe Kelly', 'Michael Bolsinger': 'Mike Bolsinger', 'Michael Montgomery': 'Mike Montgomery', 'Michael Clevinger': 'Mike Clevinger','Joseph Biagini': 'Joe Biagini', 'Luke Sims': 'Lucas Sims', 'Michael Soroka': 'Mike Soroka'}
pitchlogs['Pitcher'] = pitchlogs['Pitcher'].replace(correctnames)

In [39]:
def abbrev_name(fullname):
    beg_firstname = fullname[:3]
    lastname = fullname.split()[-1]
    abbrev_name = beg_firstname + ' ' + lastname
    return abbrev_name

RSgamelogs['HomeSPAbbrev'] = RSgamelogs['HomeStartingPitcherName'].apply(abbrev_name)
RSgamelogs['VisitSPAbbrev'] = RSgamelogs['VisitorStartingPitcherName'].apply(abbrev_name)
pitchlogs['PitchAbbrev'] = pitchlogs['Pitcher'].apply(abbrev_name)

In [40]:

pitchlogs.loc[((pitchlogs['Game Date']== '2014-03-22') & (pitchlogs['Pitcher']=='Trevor Cahill')),'Game Date']  = datetime.strptime('2014-03-23', '%Y-%m-%d')
pitchlogs.loc[((pitchlogs['Game Date']== '2018-09-12') & (pitchlogs['Pitcher']=='Jason Vargas')),'Game Date'] = datetime.strptime('2018-09-13', '%Y-%m-%d')


In [41]:
gamelogs_hsp2 = pd.merge(RSgamelogs, pitchlogs, how='left', left_on = ['Date','HomeSPAbbrev'], right_on = ['Game Date','PitchAbbrev'])
nullvals2 = gamelogs_hsp2.isnull().sum()
nullrows2= gamelogs_hsp2[gamelogs_hsp2['K'].isnull()]

print(nullrows2['HomeStartingPitcherName'].unique())
nullrows2[['Date','HomeStartingPitcherName','HomeSPAbbrev','Pitcher','HomeTeam','VisitingTeam']]

[]


,Date,HomeStartingPitcherName,HomeSPAbbrev,Pitcher,HomeTeam,VisitingTeam


In [42]:
gamelogs_hsp2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24298 entries, 0 to 24297
Columns: 184 entries, Date to PitchAbbrev
dtypes: datetime64[ns](2), float64(3), int64(90), object(89)
memory usage: 34.3+ MB


In [43]:
dups = gamelogs_hsp2[gamelogs_hsp2.duplicated(subset=['Date','HomeSPAbbrev'], keep=False)]

In [44]:
dups[['Date','Pitcher','HomeSPAbbrev', 'HomeStartingPitcherName','VisitorStartingPitcherName','HomeTeam', 'VisitingTeam']]

,Date,Pitcher,HomeSPAbbrev,HomeStartingPitcherName,VisitorStartingPitcherName,HomeTeam,VisitingTeam


In [45]:
#now merge the cleaned gamelogs which contain starting pitcher stats for the home team with the pitchlogs again
#merge them on the visiting starting pitcher this time to get the final gamelogs data frame
gamelogs_t = pd.merge(gamelogs_hsp2, pitchlogs, how='left', left_on = ['Date','VisitSPAbbrev'], right_on = ['Game Date','PitchAbbrev'], suffixes= ['_H', '_V'])


In [46]:
#check that there are no new null values
gamelogs_t.replace('',np.nan, inplace=True)
nullvals3 = gamelogs_t.isnull().sum().sort_values(ascending=False)
nullvals3[nullvals3>0]

ForfeitInfo           24298
UmpireRFID            24294
UmpireRFName          24294
UmpireLFID            24294
UmpireLFName          24294
ProtestInfo           24285
CompletionInfo        24280
AdditionalInfo        24165
SavingPitcherID       11867
SavingPitcherName     11867
GameWinningRBIID       1162
GameWinningRBIName     1162
Umpire2BID               24
Umpire2BName             24
WP_V                      9
Opponent_V                9
Game Date_V               9
Pitcher_V                 9
PitchAbbrev_V             9
GS_V                      9
HR_V                      9
CG_V                      9
Sho_V                     9
IP_V                      9
H_V                       9
R_V                       9
L_V                       9
W_V                       9
GF_V                      9
ER_V                      9
Sv_V                      9
BB_V                      9
K_V                       9
BK_V                      9
HBP_V                     9
LosingPitcherID     

These are the same null values we had originally with just the RetroSheet game logs.

In [47]:
nullrows= gamelogs_t[gamelogs_t['K_V'].isnull()]

print(nullrows['VisitorStartingPitcherName'].unique())
nullrows[['Date','VisitorStartingPitcherName','VisitSPAbbrev','VisitingTeam','HomeTeam', 'VisitorPitchers','WinningPitcherName','LosingPitcherNAme']]

['Andrew Carpenter' 'Edinson Volquez' 'Roy Oswalt' 'Hyun-Jin Ryu'
 'Adrian Sampson' 'Jordan Lyles' 'Duane Underwood' 'Clay Buchholz'
 'Jeff Brigham']


,Date,VisitorStartingPitcherName,VisitSPAbbrev,VisitingTeam,HomeTeam,VisitorPitchers,WinningPitcherName,LosingPitcherNAme
536,2009-05-16,Andrew Carpenter,And Carpenter,PHI,WAS,2,Andrew Carpenter,Daniel Cabrera
5144,2011-04-22,Edinson Volquez,Edi Volquez,CIN,SLN,5,Kyle McClellan,Matt Maloney
6719,2011-08-19,Roy Oswalt,Roy Oswalt,PHI,WAS,5,Todd Coffey,Ryan Madson
12151,2014-03-23,Hyun-Jin Ryu,Hyu Ryu,LAN,ARI,8,Hyun-Jin Ryu,Trevor Cahill
18089,2016-06-23,Adrian Sampson,Adr Sampson,SEA,DET,7,Kyle Ryan,Steve Cishek
22998,2018-06-23,Jordan Lyles,Jor Lyles,SDN,SFN,6,Reyes Moronta,Adam Cimber
23023,2018-06-25,Duane Underwood,Dua Underwood,CHN,LAN,5,Kenta Maeda,Duane Underwood
24052,2018-09-13,Clay Buchholz,Cla Buchholz,ARI,COL,8,Kyle Freeland,Matt Koch
24054,2018-09-13,Jeff Brigham,Jef Brigham,FLO,NYN,5,Jason Vargas,Jeff Brigham


Reasons for the null values: <br>
Andrew Carpenter is listed as Drew Carpenter in baseball musings. <br>
For the 2011-04-24, BaseballMusings lists Matt Maloney as the starter on 2011-04-22.  Edinson Volquez was scheduled to start but, the Reds stared Maloney instead after a rain delay.  <br>
Roy Oswalt was listed as starting 8/19/2011 and 8/20/2011 in RetroSheet, but Kyle Kendrick actually started on 8/19/2011.  <br>
Hyun-Jun Ryu started on 3-23-2014 (Australian local time) but it was listed as 3-22-2014 on Baseball Musings. <br>
Vidal Nuno started on 6-23-2016 for Seattle, since "pitcher Adrian Sampson exited with discomfort in his right elbow while warming up for the first inning" http://m.espn.com/mlb/gamecast?gameId=360623106&wjb=.  <br>
On 6-23-2018, Jordan Lyle scratched so Matt Straham started for the Padres. <br>
Duane Underwood is listed as Duane Underwood Jr. on Baseball Musings. <br>
Matt Koch started for the Diamondbacks on 9-13-2018 because Clay Buchholtz scratched. <br>
Jeff Brigham pitched on 9-13-2018 but was listed as 9-12-2018 on Baseball Musings. <br>

In the cases where the starting pitcher is wrong, I will correct it in the RSGameLogs and I will subtract one for the number of visiting pitchers used ('VisitorPitchers') because I do not want to count the pitcher who scratched 

In [48]:
#Fix the issues

#Correct wrong dates in pitchlogs
pitchlogs.loc[((pitchlogs['Game Date']== '2014-03-22') & (pitchlogs['Pitcher']=='Hyun-Jin Ryu')),'Game Date']  = datetime.strptime('2014-03-23', '%Y-%m-%d')
pitchlogs.loc[((pitchlogs['Game Date']== '2018-09-12') & (pitchlogs['Pitcher']=='Jeff Brigham')),'Game Date'] = datetime.strptime('2018-09-13', '%Y-%m-%d')

#correct abbreviated names manually
pitchlogs.loc[pitchlogs['Pitcher'] == 'Drew Carpenter','Pitcher'] = 'Andrew Carpenter'
pitchlogs.loc[pitchlogs['Pitcher'] == 'Duane Underwood Jr.','Pitcher'] = 'Duane Underwood'

#correct the starting pitcher in RS GameLogs to refelct who actually started and not who was scheduled to start
VSP_idx = RSgamelogs.columns.get_loc('VisitorStartingPitcherName')
NVP_idx = RSgamelogs.columns.get_loc('VisitorPitchers')

idxEV = RSgamelogs.loc[(RSgamelogs['VisitorStartingPitcherName']=='Edinson Volquez') & (RSgamelogs['Date']=='2011-04-22'), :].index
RSgamelogs.iloc[idxEV, NVP_idx] -= 1
RSgamelogs.iloc[idxEV, VSP_idx] ='Matt Maloney'

idxRO = RSgamelogs.loc[(RSgamelogs['VisitorStartingPitcherName']=='Roy Oswalt') & (RSgamelogs['Date']=='2011-08-19')].index
RSgamelogs.iloc[idxRO, NVP_idx] -= 1
RSgamelogs.iloc[idxRO, VSP_idx] ='Kyle Kendrick'


idxAS = RSgamelogs.loc[(RSgamelogs['VisitorStartingPitcherName']=='Adrian Sampson') & (RSgamelogs['Date']=='2016-06-23')].index
RSgamelogs.iloc[idxAS, NVP_idx] -= 1
RSgamelogs.iloc[idxAS, VSP_idx] = 'Vidal Nuno'

idxJL = RSgamelogs.loc[(RSgamelogs['VisitorStartingPitcherName']=='Jordan Lyles') & (RSgamelogs['Date']=='2018-06-23')].index
RSgamelogs.iloc[idxJL, NVP_idx] -= 1
RSgamelogs.iloc[idxJL, VSP_idx] = 'Matt Strahm'

idxCB = RSgamelogs.loc[(RSgamelogs['VisitorStartingPitcherName']=='Clay Buchholz') & (RSgamelogs['Date']=='2018-09-13')].index
RSgamelogs.iloc[idxCB, NVP_idx] -= 1
RSgamelogs.iloc[idxCB, VSP_idx] = 'Matt Koch'


In [49]:
#reset abbreviations
RSgamelogs['HomeSPAbbrev'] = RSgamelogs['HomeStartingPitcherName'].apply(abbrev_name)
RSgamelogs['VisitSPAbbrev'] = RSgamelogs['VisitorStartingPitcherName'].apply(abbrev_name)
pitchlogs['PitchAbbrev'] = pitchlogs['Pitcher'].apply(abbrev_name)

In [50]:
#remerge the dataframes
newgamelogs_hsp = pd.merge(RSgamelogs, pitchlogs, how='left', left_on = ['Date', 'HomeSPAbbrev'], right_on=['Game Date', 'PitchAbbrev']) 
gamelogs = pd.merge(newgamelogs_hsp, pitchlogs, how='left', left_on = ['Date', 'VisitSPAbbrev'], right_on=['Game Date', 'PitchAbbrev'], suffixes = ['_H', '_V'])

In [51]:
gamelogs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24298 entries, 0 to 24297
Columns: 205 entries, Date to PitchAbbrev_V
dtypes: datetime64[ns](3), float64(4), int64(99), object(99)
memory usage: 38.2+ MB


In [52]:
nullcounts = gamelogs.isnull().sum().sort_values(ascending=False)
nullcounts[nullcounts>0]

ForfeitInfo           24298
UmpireRFID            24294
UmpireRFName          24294
UmpireLFID            24294
UmpireLFName          24294
ProtestInfo           24285
CompletionInfo        24280
AdditionalInfo        24165
SavingPitcherName     11867
SavingPitcherID       11867
GameWinningRBIName     1162
GameWinningRBIID       1162
Umpire2BName             24
Umpire2BID               24
WinningPitcherID          1
WinningPitcherName        1
LosingPitcherID           1
LosingPitcherNAme         1
dtype: int64

In [53]:
nullrows= gamelogs[gamelogs['K_V'].isnull()]

print(nullrows['VisitorStartingPitcherName'].unique())
nullrows[['Date','VisitorStartingPitcherName','VisitSPAbbrev','VisitingTeam','HomeTeam', 'VisitorPitchers']]


[]


,Date,VisitorStartingPitcherName,VisitSPAbbrev,VisitingTeam,HomeTeam,VisitorPitchers


In [54]:
gamelogs.head()

,Date,DoubleHeader,DayOfWeek,VisitingTeam,VisitingTeamLeague,VisitingTeamGameNumber,HomeTeam,HomeTeamLeague,HomeTeamGameNumber,VisitorRunsScored,...,H_V,R_V,ER_V,HR_V,BB_V,K_V,HBP_V,BK_V,WP_V,PitchAbbrev_V
0,2009-04-05,0,Sun,ATL,NL,1,PHI,NL,1,4,...,2,0,0,0,0,4,0,0,0,Der Lowe
1,2009-04-06,0,Mon,COL,NL,1,ARI,NL,1,8,...,7,6,6,2,1,2,0,0,0,Aar Cook
2,2009-04-06,0,Mon,NYN,NL,1,CIN,NL,1,2,...,3,1,1,0,4,7,0,0,0,Joh Santana
3,2009-04-06,0,Mon,WAS,NL,1,FLO,NL,1,6,...,6,6,6,2,0,1,0,0,0,Joh Lannan
4,2009-04-06,0,Mon,CHN,NL,1,HOU,NL,1,4,...,5,1,1,0,3,6,0,0,0,Car Zambrano


The dataframe is ready to pickle.

In [55]:
#pickle the dataframe
outfile = open('gamelogs09_18', 'wb')
pickle.dump(gamelogs, outfile)
outfile.close()

# Summary

I have imported RetroSheet Game Logs and the Starting Pitcher Logs from Baseball Musings for the 2009-2018 regular seasons.

Both datasets have been cleaned and I merged them together twice, the first time to add starting pitcher info for the home team and the second merge, was to add starting pitcher information for the visiting team.

I now have a cleaned gamelogs dataframe with 24,298 rows each representing a regular season game.

Categorial variables are :
VisitorTeamLeague (AL/NL), HomeTeamLeague, DayOfWeek, DayNight, ProtestInfo, and the Visitor and Home Positions for each player
Game Started (GS), Complete Game (CG), Game Finished (GF), Win (W), Loss (L), Save (SV), and Shutout (Sho)- for the home starting pitcher and the visiting starting pitcher.

I will now move on to the Data Storytelling section...